In [0]:
%run ../oreilly-databricks-dea/Includes/School-Setup

In [0]:
%sql
CREATE TABLE students_json AS
SELECT * FROM json.`${dataset.school}/students-json`

In [0]:
%sql
SELECT * FROM students_json;

In [0]:
%sql
SELECT * FROM csv.`${dataset.school}/courses-csv`

In [0]:
%sql
CREATE TABLE courses_csv
(course_id STRING, title STRING, instructor STRING, category STRING, price DOUBLE)
USING csv
OPTIONS (
  header "true",
  delimiter ";"
)
LOCATION "${dataset.school}/courses-csv" ;

In [0]:
%sql
SELECT * FROM courses_csv;

In [0]:
%sql
DESCRIBE EXTENDED courses_csv

In [0]:
%fs ls dbfs:/mnt/DE-Associate-Book/datasets/school/courses-csv

In [0]:
%sql
SELECT student_id, email, profile:first_name, profile:last_name
FROM students_json

In [0]:
%sql
CREATE TABLE IF NOT EXISTS enrollments 
USING parquet
LOCATION "/mnt/DE-Associate-Book/datasets/school/enrollments"

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW enrollments_view AS
SELECT enroll_id, student_id, explode(courses) as course FROM enrollments

In [0]:
%sql
SELECT student_id, array_distinct(collect_set(enroll_id)) as enrollments, collect_set(course) 
FROM enrollments_view
GROUP BY student_id

In [0]:
%sql
CREATE TABLE IF NOT EXISTS enrollments_update
USING parquet
LOCATION "/mnt/DE-Associate-Book/datasets/school/enrollments-new"

In [0]:
%sql
SELECT * FROM enrollments_update
    UNION
SELECT * FROM enrollments

In [0]:
%sql
SELECT * FROM enrollments_update
    INTERSECT
SELECT * FROM enrollments

In [0]:
%sql
SELECT * FROM enrollments_update
    MINUS
SELECT * FROM enrollments

In [0]:
%sql
SELECT student_id, email, from_json(profile, schema_of_json('{"first_name": "string", "last_name": "string"}')) as profil_struct
FROM students_json

In [0]:
%sql
SELECT * FROM (
SELECT enroll_id, student_id, course.course_id as course_id, course.discount_percent as discount, course.subtotal as subtotal
FROM enrollments_view
)
PIVOT (
  SUM(subtotal) FOR course_id IN ('C01', 'C02', 'C03')
  )


In [0]:
%sql
SELECT email, EXPLODE(TRANSFORM(array(gpa), x -> x * 2)) as gpa_double 
FROM students_json

In [0]:
%sql
CREATE OR REPLACE FUNCTION gpa_double(gpa DOUBLE)
RETURNS INT
RETURN CASE 
  WHEN gpa > 2 THEN CAST(ROUND(gpa * -2) AS INT)
  ELSE CAST(ROUND(gpa * 2) AS INT)
  END;

DESCRIBE FUNCTION EXTENDED gpa_double

In [0]:
%sql
SELECT email, gpa, gpa_double(gpa) as gpa_double 
FROM students_json